<h4>Trabajo final</h4>

In [13]:
# dependencias iniciales
import cv2;
import mediapipe as mp;
import numpy as np;
from datetime import datetime as dt;

# ML
import pandas as pd;
from sklearn.model_selection import train_test_split;
from sklearn.linear_model import LogisticRegression;
from sklearn.svm import SVC;

In [3]:
#Importamos el dataset para iniciar el análisis
dt_entrenador = pd.read_csv("dataset/datasetFinal.csv");

################################# MODELO de ML
#Separo todos los datos con las características y las etiquetas o resultados
X = np.array(dt_entrenador.drop(['resultado'], 1));
y = np.array(dt_entrenador['resultado']);

#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2);
print('Son {} datos para entrenamiento y {} datos para prueba'.format(X_train.shape[0], X_test.shape[0]));


#Modelo de Máquinas de Vectores de Soporte
algoritmo = SVC();
algoritmo.fit(X_train, y_train);
Y_pred = algoritmo.predict(X_test);
print('Accuracy Máquinas de Vectores de Soporte: {}'.format(algoritmo.score(X_train, y_train)));

C:\Users\inFAM\AppData\Local\Temp\ipykernel_8832\4180259612.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(dt_entrenador.drop(['resultado'], 1));


Son 140000 datos para entrenamiento y 35000 datos para prueba
Accuracy Máquinas de Vectores de Soporte: 1.0


In [4]:
# inicializando media pipe
mp_drawing = mp.solutions.drawing_utils;
mp_pose = mp.solutions.pose;

In [5]:
'''
Funcion que calcula un angulo en grados sexagecimales a partir de tres puntos
@param puntoA coordenadas [x,y] que define el primer punto o punto inicial
@param puntoB coordenadas [x,y] que define el segundo punto o punto intermedio
@param puntoC coordenadas [x,y] que define el tercer punto o punto final    
@return El angulo calculado entre interseccion de las lineas formadas por (puntoA,puntoB) y (puntoB,puntoC)
'''
def calcularAngulo(puntoA,puntoB,puntoC):

    # convirtiendo a npArray: [x,y]
    a = np.array(puntoA);# primer punto
    b = np.array(puntoB);# punto medio
    c = np.array(puntoC);# punto final

    # calculando el angulo en radianes
    rad = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0]);
    
    # convirtiendo radianes a grados sexagecimales
    angulo = np.abs(rad*180.0/np.pi);
    
    # si el angulo es mayor que 180 que devuelva el complemento hasta 360
    if angulo > 180.0 :
        angulo = 360 - angulo;
        
    return round(angulo, 4);

<h4>Abduccion inicial</h4>

In [6]:
# realizando la captura de angulo de la abduccion inicial en imagen

# inicializando una lista de angulos de la ejecucion de la abduccion:
angulos_abduccion = [];
# leyendo imagenes estaticas
with mp_pose.Pose(static_image_mode=True) as pose:
    imagen = cv2.imread("abducion.jpg");
    
    #capturando las dimenciones  de la imagen:
    alto = imagen.shape[0];
    ancho = imagen.shape[1];
    #print(f'ancho y alto {ancho},{alto}')
    
    #trasnformando la imagen a rgb para ser tratado por opencv:
    imagen_rgb = cv2.cvtColor(imagen,cv2.COLOR_BGR2RGB);

    imagen_rgb.flags.writeable = False; # ---
    
    # detecciones de pin points:
    resultados = pose.process(imagen_rgb);
    
    # mostrando los 33 puntos de referencia
    #print('pose landmarks:',resultados.pose_landmarks)


    # devolviendo la imagen del fotograma a bgr: ----
    imagen_rgb.flags.writeable = True;
    imagen_rgb = cv2.cvtColor(imagen_rgb,cv2.COLOR_RGB2BGR);
    
    
    #si hay detecciones
    if resultados.pose_landmarks is not None:

        puntos = resultados.pose_landmarks.landmark;
        #print(f'Numero de puntos de referencia :{len(puntos)}');

        ### calculando el angulo de la abduccion:

        ### 1 abduccion inicial ángulo de apertura de cadera en posición recta: (26,24,23)  
        # 26
        x1 = puntos[mp_pose.PoseLandmark.RIGHT_KNEE].x;
        y1 = puntos[mp_pose.PoseLandmark.RIGHT_KNEE].y;
        
        # 24
        x2 = puntos[mp_pose.PoseLandmark.RIGHT_HIP].x;
        y2 = puntos[mp_pose.PoseLandmark.RIGHT_HIP].y;
        
        # 23
        x3 = puntos[mp_pose.PoseLandmark.LEFT_HIP].x;
        y3 = puntos[mp_pose.PoseLandmark.LEFT_HIP].y;
        
        # calculando el angulo 
        angulo = calcularAngulo([x1,y1],[x2,y2],[x3,y3]);

        # añadir angulo a una lista:
        angulos_abduccion.append(angulo);

        print(f'angulo arriba calculado (26,24,23):{angulo}');

        # dibujando en el video el angulo calculado en el punto medio:
        cv2.putText(imagen_rgb,
                    str(angulo.astype(int)),
                    tuple(np.multiply([x2,y2],[ancho,alto]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, 
                    (255,255,255),
                    2,
                    cv2.LINE_AA
                );


        ### 2 abduccion inicial angulo de cierre de rodilla en posición recta: (23,25,27)
        # 23
        x1 = puntos[mp_pose.PoseLandmark.LEFT_HIP].x;
        y1 = puntos[mp_pose.PoseLandmark.LEFT_HIP].y;
        
        # 25
        x2 = puntos[mp_pose.PoseLandmark.LEFT_KNEE].x;
        y2 = puntos[mp_pose.PoseLandmark.LEFT_KNEE].y;
        
        # 27
        x3 = puntos[mp_pose.PoseLandmark.LEFT_ANKLE].x;
        y3 = puntos[mp_pose.PoseLandmark.LEFT_ANKLE].y;
        
        # calculando el angulo formado entre el punto de la cadera izquierda, la rodilla izquierda y el tobillo izquierdo:
        angulo = calcularAngulo([x1,y1],[x2,y2],[x3,y3]);

        # añadir angulo a una lista:
        angulos_abduccion.append(angulo);

        print(f'angulo arriba calculado (23,25,27):{angulo}');

        # dibujando en el video el angulo calculado en el punto medio:
        cv2.putText(imagen_rgb,
                    str(angulo.astype(int)),
                    tuple(np.multiply([x2,y2],[ancho,alto]).astype(int)),
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, 
                    (255,255,255),
                    2,
                    cv2.LINE_AA
                );
        
        print("angulos de abducion: ",angulos_abduccion);
       
        #pintando los puntos de referencia(landmarks)  y las conexiones sobre la imagen:
        mp_drawing.draw_landmarks(imagen,
                                  resultados.pose_landmarks, #puntos
                                  mp_pose.POSE_CONNECTIONS, #conexiones
                                  mp_drawing.DrawingSpec(color=(128,0,250),thickness=2,circle_radius=3), # colorRGB,grosor y radio del punto
                                  mp_drawing.DrawingSpec(color=(52, 235, 116),thickness=2) #colorRGB,grosor de las lineas de conexion 
                                 );
    
        
    # pintando la imagen
    cv2.imshow("Imag",imagen);
    cv2.waitKey(0);
cv2.destroyAllWindows;

angulo arriba calculado (26,24,23):87.0685
angulo arriba calculado (23,25,27):33.2941
angulos de abducion:  [87.0685, 33.2941]


In [7]:
print("angulos de abducion: ",angulos_abduccion);

angulos de abducion:  [87.0685, 33.2941]


<h4>Video </h4>

In [8]:
# captura del video
cap = cv2.VideoCapture('sentadilla.mp4');

In [9]:
# inicializando contador de sentadillas y stage
contador = 0;
stage = None;

In [14]:
# configurando la instancia de mediapipe
with mp_pose.Pose(static_image_mode=False,min_detection_confidence = 0.5,min_tracking_confidence = 0.5) as pose:

    # mostrando la fecha y hora de ejecucion : dd/mm/YY H:M:S
    dt_string = dt.now().strftime("%d/%m/%Y %H:%M:%S")
    print("Ejecucion al : ", dt_string);
    print('========================================================================');
    Y = "...";

    
    # leyendo  el video
    while cap.isOpened() :

        ret,frame = cap.read();
        if ret == False:
            break;

        # capturando las dimenciones del frame de la imagen del video:
        alto = frame.shape[0];
        ancho = frame.shape[1];
        #print(f'ancho y alto del frame de la imagen: {ancho},{alto}')

        # trasnformando la imagen del fotograma a rgb para ser tratado por opencv:
        img_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB);
        img_frame.flags.writeable = False;
        
        # detecciones de pin points:
        resultados = pose.process(img_frame);
        
        # devolviendo la imagen del fotograma a bgr:
        img_frame.flags.writeable = True;
        img_frame = cv2.cvtColor(img_frame,cv2.COLOR_RGB2BGR);
        
        # si hay detecciones de pin points
        if resultados.pose_landmarks is not None:

            # inicializando una lista de angulos de la ejecucion de la sentadilla:
            angulos_sentadilla = [];
            angulos=[];
            
            
            # recuperando info del landmark(punto de referecencia):
            try:

                puntos = resultados.pose_landmarks.landmark;
                #print(f'Numero de puntos de referencia :{len(puntos)}');

                ### determinando si se esta arriba o abajo:

                # CADERA IZQUIERDA 23: obteniendo la coordenada x,y,z del punto de referencia (landmark)
                x1 = puntos[mp_pose.PoseLandmark.LEFT_HIP].x;
                y1 = puntos[mp_pose.PoseLandmark.LEFT_HIP].y;
                
                # RODILLA IZQUIERDA 25: obteniendo la coordenada x,y del punto de referencia (landmark) :
                x2 = puntos[mp_pose.PoseLandmark.LEFT_KNEE].x;
                y2 = puntos[mp_pose.PoseLandmark.LEFT_KNEE].y;
                
                # TOBILLO IZQUIERDO 27: obteniendo la coordenada x,y,z del punto de referencia (landmark):
                x3 = puntos[mp_pose.PoseLandmark.LEFT_ANKLE].x;
                y3 = puntos[mp_pose.PoseLandmark.LEFT_ANKLE].y;
                
                # calculando el angulo formado entre el punto de la cadera izquierda, la rodilla izquierda y el tobillo izquierdo:
                angulo = calcularAngulo([x1,y1],[x2,y2],[x3,y3]);

                # dibujando un rectagulo en el video
                cv2.rectangle(img_frame,(0,0),(400,180),(245,117,16),-1);
                
                # dibujando la palabra 'sentadilla' en el video
                cv2.putText(img_frame,
                            'sentadilla',
                            (10,20),
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            1, 
                            (255,255,255),
                            1,
                            cv2.LINE_AA
                           );



                # actualizando contador de sentadillas e invocando al modelo de ML para la clasificación:
                if angulo > 170:

                    stage = 'arriba';
                

                elif angulo < 160 and stage == 'arriba':

                    print('======================   INICIO  ABAJO  =====================');
                    stage = 'abajo';
                    contador = contador + 1;
                    

                    ### 3 aperturas de  piernas abajo: calcular distancia entre 26 y 25 y calcular angulo(por desarrollar)
                     

                    ### 4 ángulo de  cadera a espalda abajo : (11,23,25) ideal 65-45, no ideal: menores a 45 y mayores a 70
                    # 11: obteniendo la coordenada x,y,z del punto de referencia (landmark)
                    x1 = puntos[mp_pose.PoseLandmark.LEFT_SHOULDER].x;
                    y1 = puntos[mp_pose.PoseLandmark.LEFT_SHOULDER].y;

                    # 23: obteniendo la coordenada x,y del punto de referencia (landmark) :
                    x2 = puntos[mp_pose.PoseLandmark.LEFT_HIP].x;
                    y2 = puntos[mp_pose.PoseLandmark.LEFT_HIP].y;

                    # 25: obteniendo la coordenada x,y,z del punto de referencia (landmark):
                    x3 = puntos[mp_pose.PoseLandmark.LEFT_KNEE].x;
                    y3 = puntos[mp_pose.PoseLandmark.LEFT_KNEE].y;

                    # setetando coordenadas x,y para el cadera, rodilla y tobillo izquierdo: inicio,medio y fin
                    punto_inicial = [x1,y1];
                    punto_medio = [x2,y2];
                    punto_final = [x3,y3];

                    # calculando el angulo formado entre el punto de la cadera izquierda, la rodilla izquierda y el tobillo izquierdo:
                    angulo = calcularAngulo(punto_inicial,punto_medio,punto_final);

                    # añadir angulo a una lista:
                    angulos_sentadilla.append(angulo);
                    print(f'angulo abajo calculado (11,23,25):{angulo}');

                    # dibujando en el video el angulo calculado en el punto medio:
                    cv2.putText(img_frame,
                                str(angulo.astype(int)),
                                tuple(np.multiply(punto_medio,[ancho,alto]).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                0.5, 
                                (255,255,255),
                                2,
                                cv2.LINE_AA
                            );

                    ### 5 angulos de cerradura de lados abajo: (23,25,27) ideal 45-60 , no ideal: menores a 45 y mayores a 65
                    # 23: obteniendo la coordenada x,y,z del punto de referencia (landmark)
                    x1 = puntos[mp_pose.PoseLandmark.LEFT_HIP].x;
                    y1 = puntos[mp_pose.PoseLandmark.LEFT_HIP].y;

                    # 25: obteniendo la coordenada x,y del punto de referencia (landmark) :
                    x2 = puntos[mp_pose.PoseLandmark.LEFT_KNEE].x;
                    y2 = puntos[mp_pose.PoseLandmark.LEFT_KNEE].y;

                    # 27: obteniendo la coordenada x,y,z del punto de referencia (landmark):
                    x3 = puntos[mp_pose.PoseLandmark.LEFT_ANKLE].x;
                    y3 = puntos[mp_pose.PoseLandmark.LEFT_ANKLE].y;

                    # setetando coordenadas x,y para el cadera, rodilla y tobillo izquierdo: inicio,medio y fin
                    punto_inicial = [x1,y1];
                    punto_medio = [x2,y2];
                    punto_final = [x3,y3];

                    # calculando el angulo formado entre el punto de la cadera izquierda, la rodilla izquierda y el tobillo izquierdo:
                    angulo = calcularAngulo(punto_inicial,punto_medio,punto_final);

                    # añadir angulo a una lista:
                    angulos_sentadilla.append(angulo);
                    print(f'angulo abajo calculado (23,25,27):{angulo}');

                    # dibujando en el video el angulo calculado en el punto medio:
                    cv2.putText(img_frame,
                                str(angulo.astype(int)),
                                tuple(np.multiply(punto_medio,[ancho,alto]).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                0.5, 
                                (255,255,255),
                                2,
                                cv2.LINE_AA
                            );


                    ### 6 angulos de los tobillos abajo :(27,29,31)  ideal:80-85 , no ideal menor a 80  o mayor 85
                    # 27: obteniendo la coordenada x,y,z del punto de referencia (landmark)
                    x1 = puntos[mp_pose.PoseLandmark.LEFT_ANKLE].x;
                    y1 = puntos[mp_pose.PoseLandmark.LEFT_ANKLE].y;

                    # 29: obteniendo la coordenada x,y del punto de referencia (landmark) :
                    x2 = puntos[mp_pose.PoseLandmark.LEFT_HEEL].x;
                    y2 = puntos[mp_pose.PoseLandmark.LEFT_HEEL].y;

                    # 31: obteniendo la coordenada x,y,z del punto de referencia (landmark):
                    x3 = puntos[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x;
                    y3 = puntos[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y;

                    # setetando coordenadas x,y para el cadera, rodilla y tobillo izquierdo: inicio,medio y fin
                    punto_inicial = [x1,y1];
                    punto_medio = [x2,y2];
                    punto_final = [x3,y3];

                    # calculando el angulo formado entre el punto de la cadera izquierda, la rodilla izquierda y el tobillo izquierdo:
                    angulo = calcularAngulo(punto_inicial,punto_medio,punto_final);

                    # añadir angulo a una lista:
                    angulos_sentadilla.append(angulo);
                    print(f'angulo abajo calculado (27,29,31) :{angulo}');

                    # dibujando en el video el angulo calculado en el punto medio:
                    cv2.putText(img_frame,
                                str(angulo.astype(int)),
                                tuple(np.multiply(punto_medio,[ancho,alto]).astype(int)),
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                0.5, 
                                (255,255,255),
                                2,
                                cv2.LINE_AA
                            );


                    print(contador);


                    # pintando el resultado de cada ejecucion: angulos_abduccion y angulos_sentadilla
                    angulos = angulos_abduccion + angulos_sentadilla;
                    print("angulos en lista :",angulos);

                    print('======================   FIN ABAJO  =====================');

            
                # invocando el modelo de ML
                if (len(angulos)>0):
                    X = [angulos];
                    print("valor  a predecir : :",X);
                    Y = algoritmo.predict(X);
                    #Y = "PPPPPPPP";
                    print("Prediccion : :",Y);

                # dibujando el contador en el video  
                cv2.putText(img_frame,
                            str(contador),
                            (10,80),
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2, 
                            (255,255,255),
                            2,
                            cv2.LINE_AA
                        );

                # dibujando si la ejecucion fue correcta o incorrecta
                cv2.putText(img_frame,
                            Y[0],
                            (10,140),
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            2, 
                            (255,255,255),
                            2,
                            cv2.LINE_AA
                        );

            except:
                pass;

            
            # pintando los puntos de referencia(landmarks)  y las conexiones sobre la imagen del video:
            mp_drawing.draw_landmarks(img_frame,
                                      resultados.pose_landmarks, # puntos
                                      mp_pose.POSE_CONNECTIONS, # conexiones
                                      mp_drawing.DrawingSpec(color=(128,0,250),thickness=2,circle_radius=3), # colorRGB,grosor y radio del punto
                                      mp_drawing.DrawingSpec(color=(52, 235, 116),thickness=2) #colorRGB,grosor de las lineas de conexion 
                                     );

        cv2.imshow("MediaPipe Feed",img_frame);

        if cv2.waitKey(10) &  0xFF == ord('q'):
            break;
    


    #liberando el dispositivo de captura de video            
    cap.release();
    cv2.destroyAllWindows;

Ejecucion al :  23/10/2023 23:30:08
======================   INICIO  ABAJO  =====================
angulo abajo calculado (11,23,25):166.0106
angulo abajo calculado (23,25,27):159.6617
angulo abajo calculado (27,29,31) :35.5685
1
angulos en lista : [87.0685, 33.2941, 166.0106, 159.6617, 35.5685]
======================   FIN ABAJO  =====================
valor  a predecir : : [[87.0685, 33.2941, 166.0106, 159.6617, 35.5685]]
Prediccion : : ['INCORRECTO']
======================   INICIO  ABAJO  =====================
angulo abajo calculado (11,23,25):166.7103
angulo abajo calculado (23,25,27):155.5961
angulo abajo calculado (27,29,31) :37.9053
2
angulos en lista : [87.0685, 33.2941, 166.7103, 155.5961, 37.9053]
======================   FIN ABAJO  =====================
valor  a predecir : : [[87.0685, 33.2941, 166.7103, 155.5961, 37.9053]]
Prediccion : : ['INCORRECTO']
======================   INICIO  ABAJO  =====================
angulo abajo calculado (11,23,25):150.031
angulo abajo calcula

In [7]:
display(tuple(np.multiply(rodilla_izq,[ancho,alto]).astype(int)));

(700, 487)

In [8]:
display(tuple(np.multiply(rodilla_izq,[640,480]).astype(int)));

(349, 319)

In [9]:
display(resultados.pose_landmarks);

landmark {
  x: 0.49372196197509766
  y: 0.21129760146141052
  z: -0.258330762386322
  visibility: 0.999973714351654
}
landmark {
  x: 0.499051570892334
  y: 0.19760151207447052
  z: -0.25802209973335266
  visibility: 0.9999450445175171
}
landmark {
  x: 0.5023015737533569
  y: 0.197040855884552
  z: -0.25804710388183594
  visibility: 0.9999483227729797
}
landmark {
  x: 0.5059512853622437
  y: 0.19638150930404663
  z: -0.2581078112125397
  visibility: 0.9999539256095886
}
landmark {
  x: 0.49414247274398804
  y: 0.19833505153656006
  z: -0.2436162382364273
  visibility: 0.9999425411224365
}
landmark {
  x: 0.4930489957332611
  y: 0.19839777052402496
  z: -0.24361473321914673
  visibility: 0.9999390244483948
}
landmark {
  x: 0.4917967617511749
  y: 0.19855014979839325
  z: -0.24361537396907806
  visibility: 0.9999396204948425
}
landmark {
  x: 0.5181424617767334
  y: 0.20325390994548798
  z: -0.20797845721244812
  visibility: 0.9999443888664246
}
landmark {
  x: 0.4971415400505066
  y

In [10]:
display(mp_pose.POSE_CONNECTIONS);

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [ ]:
# especificacion documentacion de  mp_drawing.DrawingSpec:
mp_drawing.DrawingSpec??

<h3>puntos</h3>

<img src="https://camo.githubusercontent.com/7fbec98ddbc1dc4186852d1c29487efd7b1eb820c8b6ef34e113fcde40746be2/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f706f73655f747261636b696e675f66756c6c5f626f64795f6c616e646d61726b732e706e67" >

In [11]:
#impriendo los lansmakrs
i = 0;
for lndmrk in mp_pose.PoseLandmark:
    print(f'{i} ==> {lndmrk.name}');
    i = i+1;
    

0 ==> NOSE
1 ==> LEFT_EYE_INNER
2 ==> LEFT_EYE
3 ==> LEFT_EYE_OUTER
4 ==> RIGHT_EYE_INNER
5 ==> RIGHT_EYE
6 ==> RIGHT_EYE_OUTER
7 ==> LEFT_EAR
8 ==> RIGHT_EAR
9 ==> MOUTH_LEFT
10 ==> MOUTH_RIGHT
11 ==> LEFT_SHOULDER
12 ==> RIGHT_SHOULDER
13 ==> LEFT_ELBOW
14 ==> RIGHT_ELBOW
15 ==> LEFT_WRIST
16 ==> RIGHT_WRIST
17 ==> LEFT_PINKY
18 ==> RIGHT_PINKY
19 ==> LEFT_INDEX
20 ==> RIGHT_INDEX
21 ==> LEFT_THUMB
22 ==> RIGHT_THUMB
23 ==> LEFT_HIP
24 ==> RIGHT_HIP
25 ==> LEFT_KNEE
26 ==> RIGHT_KNEE
27 ==> LEFT_ANKLE
28 ==> RIGHT_ANKLE
29 ==> LEFT_HEEL
30 ==> RIGHT_HEEL
31 ==> LEFT_FOOT_INDEX
32 ==> RIGHT_FOOT_INDEX


: 